# Coursework 2: Data Processing

## Task 1
This coursework will assess your understanding of using NoSQL to store and retrieve data.  You will perform operations on data from the Enron email dataset in a MongoDB database, and write a report detailing the suitability of different types of databases for data science applications.  You will be required to run code to answer the given questions in the Jupyter notebook provided, and write a report describing alternative approaches to using MongoDB.

Download the JSON version of the Enron data (using the “Download as zip” to download the data file from http://edshare.soton.ac.uk/19548/, the file is about 380MB) and import into a collection called messages in a database called enron.  You do not need to set up any authentication.  In the Jupyter notebook provided, perform the following tasks, using the Python PyMongo library.

Answers should be efficient in terms of speed.  Answers which are less efficient will not get full marks.

In [ ]:
#########################################Read Me#########################################
#I used Mongodb 3.2.11 provoided by lab tutorial, many new features (Aggregation Pipeline Operators) can not be used like $split,$trim,$dateFromString.
#In terms of date, I can not find alternative way to add a new filed to convert the date with string format into date type, so I have to update the original 
#date field(of course modify original database is not a suggested way).
#So please run the question 5 first than question 4 and question 8.
#In terms of $split, I have to handle the logic using loop in python. There is no doubt all these increase the complexity and difficulty of the questions and may cause less efficient due to loop. 
#Please consider the curmanstance as appropriate.
#I intended to install the latest version of Mongodb originally, but there are some problem, I was afraid of wasting much time in fixing the problem, so I chose the provided environment finally. 
#And I think it’s fair for everyone to do their coursework with the same version. The lab tutorial provided us the same environment.


In [1]:
import pymongo
from pymongo import MongoClient
from datetime import datetime
from pprint import pprint
import re
import string
import dateutil
import pandas as pd

### 1)
Write a function which returns a MongoDB connection object to the "messages" collection. [4 points] 

In [2]:
def get_collection():
    """
    Connects to the server, and returns a collection object
    of the `messages` collection in the `enron` database
    """
    # YOUR CODE HERE
    from pymongo import MongoClient
    client = MongoClient('mongodb://localhost:27017')
#     print(client.list_database_names())
    db=client.enron
    collection=db.messages
    return collection


In [3]:
collection=get_collection()

### 2)

Write a function which returns the amount of emails in the messages collection in total. [4 points] 

In [177]:
def get_amount_of_messages(collection):
    """
    :param collection A PyMongo collection object
    :return the amount of documents in the collection
    """    
    # YOUR CODE HERE
    collect_amount=collection.count_documents({})
    return collect_amount


In [178]:
print('The amount of messages is: '+ str(get_amount_of_messages(collection)))

The amount of messages is: 501513


### 3) 

Write a function which returns each person who was BCCed on an email.  Include each person only once, and display only their name according to the X-To header. [4 points] 



In [179]:
def get_bcced_people(collection):
    """
    :param collection A PyMongo collection object
    :return the names of the people who have received an email by BCC
    """    
    # YOUR CODE HERE
  
    filter={"$and":[{"headers.Bcc":{"$exists":True}},{'headers.X-bcc':{'$ne':''}}]}
    cursor=collection.find(filter)
    x_bcc_list=cursor.distinct('headers.X-bcc')
    regx=r'([A-Za-z\s]+[\,]?\s[A-Za-z\s]+\.*\s\<.+\>$)+(^\'\w+([-+.]\w+)*\@\w+([-.]\w+)*\.\w+([-.]\w+)*\'$)*' #name<>,and email, actually after filter doesn't conatin email format
    re_regx1=re.compile(regx)
    regx2=r'^\'\w+([-+.]\w+)*\@\w+([-.]\w+)*\.\w+([-.]\w+)*\'$' #only email
    re_regx2=re.compile(regx2)
    print('The total rows of documents which contains BCC fields is: '+str(collection.count_documents(filter)))
    name_list=[]
    for value in x_bcc_list:
        bcc_value_list=[]
        m1=re.match(re_regx1,value)
        m2=re.match(re_regx2,value)
        if(m1):
            split_value_list=re.split('>,',value) #string list split by '>,' in one row
            for bcc_list in split_value_list: 
                bcc_value=bcc_list.split('<')[0].strip()
                name_list.append(bcc_value)
        elif(m2):
            split_value=re.split('@',value)[0]
            bcc_value=split_value[1:].strip()
            name_list.append(bcc_value)
    name_list2 = sorted(set(name_list),key= name_list.index)  #keep the original order
    return name_list2
    

In [180]:
name_list=get_bcced_people(collection)
print('The amount of the people who have received an email by BCC is: '+str(len(name_list)))
pprint(name_list)
  

The total rows of documents which contains BCC fields is: 58
The amount of the people who have received an email by BCC is: 53
[u'Beck, Sally',
 u'Denny, Jennifer',
 u'Choate, Heather',
 u'Patti Thompson',
 u'Beth Apollo',
 u'Greg Piper',
 u'Carr, James',
 u'Ratliff, Dale',
 u'Robertson, Audrey',
 u'Storey, Geoff',
 u'Germany, Chris',
 u'Parks, Joe',
 u'Dinari, Sabra L.',
 u'Sanchez, Christina',
 u'Arora, Harry',
 u'Daily, Pamela',
 u'Steve Hotte',
 u'Audrey Robertson',
 u'Batista, Daniel',
 u'Schaffer, Brian',
 u'Barrow, Cynthia',
 u'Rahaim, Christian',
 u'Phil Lowry',
 u'Gadd, Eric',
 u'Schoolcraft, Darrell',
 u'Cebryk, Doug',
 u'Eisenstein, Arnold L.',
 u'Issler, Paulo',
 u'Lee, Bob',
 u'Kaminski, Vince J',
 u'Margaret Daffin',
 u'Jason Sokolov',
 u'Whalley, Greg',
 u'Evans, Carolyn',
 u'Harris, Steven',
 u'Kilmer III, Robert',
 u'Schwieger, Jim',
 u'Majed Nachawati',
 u'Davis, Mark Dana',
 u'Sturm, Fletcher J.',
 u'Benson, Robert',
 u'Castano, Marianne',
 u'Wright, Alice',
 u'Lagra

### 4)

Write a function with parameter subject, which gets all emails in a thread with that parameter, and orders them by date (ascending). “An email thread is an email message that includes a running list of all the succeeding replies starting with the original email.”, check for detail descriptions at https://www.techopedia.com/definition/1503/email-thread [4 points]

In [147]:
def get_emails_in_thread(collection, subject):
    """
    :param collection A PyMongo collection object
    :return All emails in the thread with that subject
    """    
    # YOUR CODE HERE  
    filter={"headers.Date": {"$exists": True, "$type": 2 }}
    cursor=collection.find(filter)
    for c in cursor:
        collection.update_one({"_id" : c['_id']},{"$set" : {"headers.Date" : dateutil.parser.parse(c['headers']['Date'])}})
        
    subject_regex=re.compile(r'.*'+subject+r'.*')
    match = {
            "$match": {'headers.Subject': {'$regex':subject_regex}}
    }
    sort = {
             '$sort': {'headers.Date': pymongo.ASCENDING}
    }
    pipeline=[match,sort]
    cursor = collection.aggregate(pipeline)
    return cursor


In [148]:
subject=r'Plays and other information'
cursor=get_emails_in_thread(collection, subject)
for c in cursor:
   pprint(c)

 

{u'_id': ObjectId('4f16fc98d1e2d32371004365'),
 u'body': u"I know I might seem a little obsessed with this, but here are some of our \nplays from our flag football league.  There are plays on both tabs.\nI was thinking that we would start with me at left flanker, Paul Broderick at \ncenter, Paul Burkhart at right slot, Steve at right flanker, Sarah at \nblocker, and either Mike or Chris at blocker.  Let me know if you have \nobjections.  Also, I am sending this to Paul Burkhart so I am assuming that \nhe will play (P. Broderick can you confirm this?).\n\nSteve - if you have some plays from your team that you think might work let \nme know.\n\nLet me know what you think.\n\nHook 'Em!\n-Eric\n\n\n",
 u'filename': u'77.',
 u'headers': {u'Content-Transfer-Encoding': u'7bit',
              u'Content-Type': u'text/plain; charset=us-ascii',
              u'Date': datetime.datetime(2000, 11, 14, 12, 17),
              u'From': u'eric.bass@enron.com',
              u'Message-ID': u'<2202976.107

### 5)

Write a function which returns the percentage of emails sent on a weekend (i.e., Saturday and Sunday) as a `float` between 0 and 1. [6 points]

In [46]:
def get_percentage_sent_on_weekend(collection):
    """
    :param collection A PyMongo collection object
    :return A float between 0 and 1
    """    
    # YOUR CODE HERE
    date_regex=re.compile(r'^Sat|Sun')
    filter={"headers.Date":{"$regex":date_regex}}
    count=collection.count_documents(filter)
    amount=collection.count_documents({})
#     percentage=round(float(count)/amount,6)
    percentage=float(count)/amount
    return percentage
    pass

In [47]:
get_percentage_sent_on_weekend(collection)

0.04005479419277267

### 6)

Write a function with parameter limit. The function should return for each email account: the number of emails sent, the number of emails received, and the total number of emails (sent and received). Use the following format: [{"contact": "michael.simmons@enron.com", "from": 42, "to": 92, "total": 134}] and the information contained in the To, From, and Cc headers. Sort the output in descending order by the total number of emails. Use the parameter limit to specify the number of results to be returned. If limit is null, the function should return all results. If limit is higher than null, the function should return the number of results specified as limit. limit cannot take negative values. [10 points]

In [30]:
def get_emails_between_contacts(collection, limit):
    """
    Shows the communications between contacts
    Sort by the descending order of total emails using the To, From, and Cc headers.
    :param `collection` A PyMongo collection object    
    :param `limit` An integer specifying the amount to display, or
    if null will display all outputs
    :return A list of objects of the form:
    [{
        'contact': <<Another email address>>
        'from': 
        'to': 
        'total': 
    },{.....}]
    """    
    # YOUR CODE HERE
  
    if(not(isinstance(limit, int))):
        return False
    else:
        if(limit<0):
            return False
        else:
            flag=0
    if(limit==None):   #all
        flag=1
    
    #group by From
    group = {
        '$group': {
               '_id': '$headers.From',
               'from': {'$sum': 1}    
         }
    }

    from_cursor=collection.aggregate([group],allowDiskUse = True) 

    #group by To
    unwind = {'$unwind': '$headers'}
    
    project={'$project' :{'_id':0,
             'To':'$headers.To'}
    }

    group = {
        '$group': {
               '_id': '$To',
               'to_count': {'$sum': 1}    
         }
    }
    
    cursor=collection.aggregate([unwind,project,group],allowDiskUse = True) 
    list_to=[]
    list_to_count=[]
    for c in cursor:
        t=','  
        s=c['_id']
        if(s!=None):
            result=s.find(t)>=0 
            if(result): #contains several users
                split_list=re.split(r',',s)
                for j in range(len(split_list)):
                    list_to.append(split_list[j].strip())
                    list_to_count.append(c['to_count'])
            else:
                  list_to.append(s.strip())
                  list_to_count.append(c['to_count'])


    #group by Cc
    unwind = {'$unwind': '$headers'}
    
    project={'$project' :{'_id':0,
             'Cc':'$headers.Cc'}
    }

    group = {
        '$group': {
               '_id': '$Cc',
               'cc_count': {'$sum': 1}    
         }
    }
    
    cursor=collection.aggregate([unwind,project,group],allowDiskUse = True) 
    list_cc=[]
    list_cc_count=[]
    for c in cursor:
        dict_cc={}
        t=','  
        s=c['_id']
        if(s!=None):
            result=s.find(t)>=0 
            if(result): #contains several users
                split_list=re.split(r',',s)
                for j in range(len(split_list)):
                    list_cc.append(split_list[j].strip())
                    list_cc_count.append(c['cc_count'])  
            else:
                  list_cc.append(s.strip())
                  list_cc_count.append(c['cc_count'])
        
    df_from = pd.DataFrame(list(from_cursor))
    df_from.drop_duplicates(['_id'],inplace=True)
    
    data_to={"_id":list_to,"to":list_to_count}
    data_cc={"_id":list_cc,"cc":list_cc_count}
   
    df_to = pd.DataFrame(data_to)
    df_to.drop_duplicates(['_id'],inplace=True)
    df_cc = pd.DataFrame(data_cc)
    df_cc.drop_duplicates(['_id'],inplace=True)
                           
    df_to_cc=pd.merge(df_to, df_cc, how='outer')
    df_all=df_from.merge(df_to_cc,how='outer')
    df_all.drop_duplicates(['_id'],inplace=True)
    df_all.fillna(0,inplace=True)
    df_all['to']=df_all['to']+df_all['cc']
    total=df_all['from']+df_all['to']
    df_all['total']=total
    df_all.drop('cc',axis=1,inplace=True)
    df_all.rename(columns={'_id':'contact'},inplace=True)
    df_all.sort_values(by='total',ascending=False,inplace=True)
    
    if(flag==0):
        df_all=df_all[0:limit]
        
    result=[]
    for row_index,row in  df_all.iterrows():
        row_data={}
        row_data['contact']=row['contact']
        row_data['from']=row['from'] 
        row_data['to']=row['to']
        row_data['total']=row['total']
        result.append(row_data)
    
    return result


In [31]:
limit="10"
get_emails_between_contacts(collection, limit)

False

In [32]:
limit=-2
get_emails_between_contacts(collection, limit)

False

In [33]:
limit=10
result=get_emails_between_contacts(collection, limit)
for row in result:
    pprint(row)

{'contact': u'kay.mann@enron.com',
 'from': 16732.0,
 'to': 3.0,
 'total': 16735.0}
{'contact': u'vince.kaminski@enron.com',
 'from': 14368.0,
 'to': 5.0,
 'total': 14373.0}
{'contact': u'jeff.dasovich@enron.com',
 'from': 11394.0,
 'to': 2.0,
 'total': 11396.0}
{'contact': u'pete.davis@enron.com',
 'from': 9149.0,
 'to': 2.0,
 'total': 9151.0}
{'contact': u'chris.germany@enron.com',
 'from': 8801.0,
 'to': 4.0,
 'total': 8805.0}
{'contact': u'sara.shackleton@enron.com',
 'from': 8746.0,
 'to': 4.0,
 'total': 8750.0}
{'contact': u'tana.jones@enron.com',
 'from': 8488.0,
 'to': 2.0,
 'total': 8490.0}
{'contact': u'enron.announcements@enron.com',
 'from': 8404.0,
 'to': 4.0,
 'total': 8408.0}
{'contact': u'steven.kean@enron.com',
 'from': 6759.0,
 'to': 2.0,
 'total': 6761.0}
{'contact': u'kate.symes@enron.com',
 'from': 5438.0,
 'to': 2.0,
 'total': 5440.0}


### 7)
Write a function to find out the number of senders who were also direct receivers. Direct receiver means the email is sent to the person directly, not via cc or bcc. [4 points]

In [138]:
def get_from_to_people(collection):
    """
    :param collection A PyMongo collection object
    :return the NUMBER of the people who have sent emails and received emails as direct receivers.
    """    
    # YOUR CODE HERE
    #headers.To
    group = {
       '$group': {
           '_id': '$headers.To'},
    }
    cursor=collection.aggregate([group])
    list_to=[]
    for c in cursor:
        t=','  
        s=c['_id']
        if(s!=None):
            result=s.find(t)>=0
            if(result):
                split_list=re.split(r',',s)
                for j in range(len(split_list)):
                    list_to.append(split_list[j].strip())
            else:
                list_to.append(s)
    
    #headers.From
    list_from=collection.distinct('headers.From')
    
    list_users_count=list(set(list_from).intersection(set(list_to)))
    users_count=len(list_users_count)
    return users_count


In [139]:
users_count=get_from_to_people(collection)
print('The number of senders who were also direct receivers is: '+str(users_count))

The number of senders who were also direct receivers is: 10588


### 8)
Write a function with parameters start_date and end_date, which returns the number of email messages that have been sent between those specified dates, including start_date and end_date [4 points] 

In [191]:
def get_emails_between_dates(collection, start_date, end_date):
    """
    :param collection A PyMongo collection object
    :return All emails between the specified start_date and end_date
    """    
    # YOUR CODE HERE 
    start_date=start_date+' 00:00:00'
    end_date=end_date+' 23:59:59'
    datetime1= dateutil.parser.parse(start_date)
    datetime2= dateutil.parser.parse(end_date)
    
    filter={"headers.Date": {"$exists": True, "$type": 2 }}
    cursor=collection.find(filter)
    for c in cursor:
        collection.update_one({"_id" : c['_id']},{"$set" : {"headers.Date" : dateutil.parser.parse(c['headers']['Date'])}})
    
    filter1={'headers.Date': {'$gte': datetime1}}
    filter2={'headers.Date': {'$lte': datetime2}}
    filter = {"$and":[filter1,filter2]}
    count=collection.count_documents(filter)
    return count


In [195]:
start_date='2000-11-1'
end_date='2000-11-18'
amount=get_emails_between_dates(collection,start_date,end_date)
print('The number of email messages that have been sent between those specified dates is: '+str(amount))

The number of email messages that have been sent between those specified dates is: 18684


## Task 2
This task will assess your ability to use the Hadoop Streaming API and MapReduce to process data. For each of the questions below, you are expected to write two python scripts, one for the Map phase and one for the Reduce phase. You are also expected to provide the correct parameters to the `hadoop` command to run the MapReduce process. Write down your answers in the specified cells below.

To get started, you need to download and unzip the YouTube dataset (available at http://edshare.soton.ac.uk/19547/) onto the machine where you have Hadoop installed (this should be the virtual machine provided).

To help you, `%%writefile` has been added to the top of the cells, automatically writing them to "mapper.py" and "reducer.py" respectively when the cells are run.

### 1) 
Using Youtube01-Psy.csv, find the hourly interval in which most spam was sent. The output should be in the form of a single key-value pair, where the value is a datetime at the start of the hour with the highest number of spam comments. [9 points]

In [10]:
%%writefile mapper.py
#!/usr/bin/env python
#Answer for mapper.py
import csv
import sys
import re
import string

lines = sys.stdin.readlines()
csvreader = csv.reader(lines)
datetimes=[row[2] for row in csvreader]  #get the "DATE" column
datetime_token_prefix=[]
date_reg='\d{4}[-/]\d{2}[-/]\d{2}[T]'
date_reg_exp=re.compile(date_reg)
for datetime in datetimes:
    datetime_tokens= re.split(r':', datetime)
    for datetime_token in datetime_tokens: 
#         print(datetime_token  + "\t1") 
#         if(string.find(datetime_token,'T')!=-1):
        if(re.match(date_reg_exp,datetime_token)):
            datetime_token_prefix.append(datetime_token)

for token in datetime_token_prefix:
     print(token  + "\t1") 

Overwriting mapper.py


In [11]:
%%writefile reducer.py
#!/usr/bin/env python
#Answer for reducer.py
import sys
from collections import defaultdict

# test list
datetime_input_pairs=[
    '2013-11-07T06	1',
    '2013-11-07T06	1',
    '2014-11-03T20	1',
    '2014-11-08T15	1',
    '2014-11-08T15	1',
    '2014-11-08T15	1',
    '2014-11-06T24	1']

datetime_input_pairs = sys.stdin.readlines()
def fun_get_datetime_with_most_spam(datetime_input_pairs):
    datetime_accumulator = defaultdict(lambda: 0)
    datetime_with_most_spam_dict=defaultdict(lambda: 0)
    for row in datetime_input_pairs:
        datetime_key_value_pair = row.split("\t", 1)  

        if len(datetime_key_value_pair) != 2:
            continue

        word = datetime_key_value_pair[0]
        try:
            count = int(datetime_key_value_pair[1].strip()) #exception
        except:
            return False

        datetime_accumulator[word] = datetime_accumulator[word] + count

#     for (key, value) in  datetime_accumulator.items():
#         print(key + "\t" + str(value))
    
    try:
        datetime_with_most_spam=max(datetime_accumulator,key=datetime_accumulator.get)  #exception  ##just can find one
    except:
        return False
    datetime_with_most_spam_dict["datetime_with_most_spam_dict"]=datetime_with_most_spam+str(":00:00")
    return datetime_with_most_spam_dict
    

datetime_with_most_spam_dict=fun_get_datetime_with_most_spam(datetime_input_pairs)
if(datetime_with_most_spam_dict!=False):
    for (key, value) in  datetime_with_most_spam_dict.items():
            print(key + "\t" + str(value))

Overwriting reducer.py


In [13]:
%%bash
chmod a+x mapper_psy.py reducer_psy.py
cat Youtube01-Psy.csv | ./mapper_psy.py | ./reducer_psy.py | sort

datetime_with_most_spam_dict	2014-11-08T10


In [17]:
%%bash
#Hadoop command to run the map reduce.
rm -r output

hadoop-standalone-mode.sh

hadoop jar $HADOOP_HOME/share/hadoop/tools/lib/hadoop-streaming-*.jar \
-files    mapper.py,reducer.py \
-input    Youtube01-Psy.csv \
-mapper   ./mapper.py \
-reducer  ./reducer.py \
-output output

Hadoop switched to standalone mode.


OpenJDK Server VM warning: You have loaded library /opt/hadoop-2.8.5/lib/native/libhadoop.so.1.0.0 which might have disabled stack guard. The VM will try to fix the stack guard now.
It's highly recommended that you fix the library with 'execstack -c <libfile>', or link it with '-z noexecstack'.
18/12/20 23:13:06 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
18/12/20 23:13:07 INFO Configuration.deprecation: session.id is deprecated. Instead, use dfs.metrics.session-id
18/12/20 23:13:07 INFO jvm.JvmMetrics: Initializing JVM Metrics with processName=JobTracker, sessionId=
18/12/20 23:13:07 INFO jvm.JvmMetrics: Cannot initialize JVM Metrics with processName=JobTracker, sessionId= - already initialized
18/12/20 23:13:07 INFO mapred.FileInputFormat: Total input files to process : 1
18/12/20 23:13:07 INFO mapreduce.JobSubmitter: number of splits:1
18/12/20 23:13:08 INFO mapreduce.JobSubmitter: Submitting token

In [18]:
%%bash
cat output/part-00000
#Expected key-value output format:
#hour_with_most_spam	"2013-11-10T10:00:00"
#Additional key-value pairs are acceptable, as long as the hour_with_most_spam pair is correct.

datetime_with_most_spam_dict	2014-11-08T10:00:00


### 2) 
Find all comments associated with a username (the AUTHOR field). Return a JSON array of all comments associated with that username. (This should use the data from all 5 data files: Psy, KatyPerry, LMFAO, Eminem, Shakira) [11 points]

In [2]:
%%writefile mapper2.py
#!/usr/bin/env python2.7
#Answer for mapper.py

import csv
import sys

lines = sys.stdin.readlines()
csvreader = csv.reader(lines)
headings = next(csvreader)

authorcomments=[]
for row in csvreader:
    authorcomments.append(row[1]+"\t"+row[3])
        
for i in range(len(authorcomments)):
    print(authorcomments[i])
    

Overwriting mapper2.py


In [5]:
%%writefile reducer2.py
#!/usr/bin/env python2.7
#Answer for reducer.py

import sys
from collections import defaultdict

# test list
author_comment_input_pairs=[
    'John	a',
    'Mary	b',
    'John	c',
    'Mary	d',
    'Jane	e']

author_comment_input_pairs = sys.stdin.readlines()
author_comment_accumulator = defaultdict(lambda: [])

for row in author_comment_input_pairs:
    key_value_pair = row.split('\t',1)
    if len(key_value_pair) != 2:
        continue
        
    author = key_value_pair[0]
    comment = key_value_pair[1]
    author_comment_accumulator[author].append(comment)

for (key, value) in author_comment_accumulator.items():
    print(key + "\t" + str(value))

Overwriting reducer2.py


In [6]:
%%bash
chmod a+x mapper2.py reducer2.py
cat Youtube01-Psy.csv Youtube02-KatyPerry.csv Youtube03-LMFAO.csv Youtube04-Eminem.csv Youtube05-Shakira.csv |./mapper2.py | ./reducer2.py | sort

01FLHT	['Check out this video on YouTube:<br /><br />Eminem is amazing. \xef\xbb\xbf\n']
101Tele	['yo I know nobody will probably even read this..  But I\xe2\x80\x99m gonna type it any way because i hope at least one person will  i&#39;m a rapper with a dream. I know there&#39;s like 200k of those in this world  but please check out my music and subscribe? if you want.? thanks,  I would love nothing more than to have a decent following on youtube..  if anyone? reading this could press the &quot;THUMBS UP&quot; other people will see it  just a simple button press? could make my dream come true =) Thank You\n', 'yo I know nobody will probably even read this..  but, Imma type it any way because i hope at least one person will  i&#39;m a rapper with a dream. I know there&#39;s like 200k of those in this world  but please check out my music and subscribe? if you want.? thanks,  I would love nothing more than to have a decent following on youtube..  if anyone? reading this could press the &q

In [7]:
%%bash
#Hadoop command to run the map reduce.

rm -rf output2

hadoop-standalone-mode.sh

hadoop jar $HADOOP_HOME/share/hadoop/tools/lib/hadoop-streaming-*.jar \
-files mapper2.py,reducer2.py \
-input Youtube01-Psy.csv -input Youtube02-KatyPerry.csv -input Youtube03-LMFAO.csv -input Youtube04-Eminem.csv -input Youtube05-Shakira.csv \
-mapper ./mapper2.py \
-reducer ./reducer2.py \
-output output2 \

Hadoop switched to standalone mode.


OpenJDK Server VM warning: You have loaded library /opt/hadoop-2.8.5/lib/native/libhadoop.so.1.0.0 which might have disabled stack guard. The VM will try to fix the stack guard now.
It's highly recommended that you fix the library with 'execstack -c <libfile>', or link it with '-z noexecstack'.
18/12/20 23:00:33 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
18/12/20 23:00:35 INFO Configuration.deprecation: session.id is deprecated. Instead, use dfs.metrics.session-id
18/12/20 23:00:35 INFO jvm.JvmMetrics: Initializing JVM Metrics with processName=JobTracker, sessionId=
18/12/20 23:00:35 INFO jvm.JvmMetrics: Cannot initialize JVM Metrics with processName=JobTracker, sessionId= - already initialized
18/12/20 23:00:35 INFO mapred.FileInputFormat: Total input files to process : 5
18/12/20 23:00:35 INFO mapreduce.JobSubmitter: number of splits:5
18/12/20 23:00:35 INFO mapreduce.JobSubmitter: Submitting token

In [ ]:
#Expected key-value output format:
#John Smith	["Comment 1", "Comment 2", "Comment 3", "etc."]
#Jane Doe	["Comment 1", "Comment 2", "Comment 3", "etc."]

In [9]:
import sys
import json

author='Drake Gagne'

def get_author_comment(author):
    comments={}
    with open('output2/part-00000','r') as f:
        for row in f:
            row_str=row.split('\t',1)
            comments[row_str[0]]=row_str[1]
    json_array = json.dumps({author:comments[author]},sort_keys = True, indent = 2)
    return json_array

json_array=get_author_comment(author)
comment = json.loads(json_array)
print(author+'\t'+comment[author])

Drake Gagne	['Hey guys ready for more 87 Cyphers back check out my video on YouTube. NEW ALBUM IS OUT CHECK IT OUT. MORE MUSIC TOMORROW THANKS FOR READING\n', 'I KNOW YOU MAY NOT WANT TO READ THIS BUT please do  I&#39;m 87 Cypher an 11 year old rapper I have skill people said .my stuff isn&#39;t as good as my new stuff but its good please check out my current songs comment and like thank you for reading rap is my life\n', 'Hey guys I&#39;m 87 cypher im 11 years old and Rap is my life I recently made my second album desire ep . please take a moment to check out my album on YouTube thank you very much for reading every like comment and subscription counts\n']

